<a href="https://colab.research.google.com/github/eric88525/ChineseNL2SQL/blob/master/evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==3.0.2
!pip install cn2an
!pip install pyprind

     |████████████████████████████████| 778kB 4.5MB/s 
     |████████████████████████████████| 1.1MB 13.9MB/s 
     |████████████████████████████████| 890kB 26.1MB/s 
     |████████████████████████████████| 3.0MB 51.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=bb0f2284312d7e73d34f75490cf16108072d9db9940b82e148c80608ae2802f2
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 276kB 7.7MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=c44faef57e44e19fe8484a56796695be7a6c94a9318318266eb4cd1d04fc77e9
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built PyYAML
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
!mkdir -p drive
!google-drive-ocamlfuse drive


Mounted at /content/drive
/bin/bash: google-drive-ocamlfuse: command not found


In [ ]:
%cd ./drive/My\ Drive/Colab\ Notebooks/NL2SQL_TAN

/content/drive/My Drive/Colab Notebooks/NL2SQL_TAN


In [ ]:
import json
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from transformers import *
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import cn2an
import re

# 資料來源

In [ ]:
# get all

class T2Sdataset():
  def __init__(self,train_table_path=None,train_data_path=None,test_table_path=None,test_data_path=None,dev_table_path=None,dev_data_path=None,):
    
    if train_table_path != None:
      self.train_dict = self.get_table(train_table_path)
      self.train_data = self.get_data(self.train_dict,train_data_path)

    if test_table_path != None:
      self.test_dict = self.get_table(test_table_path)
      self.test_data = self.get_data(self.test_dict,test_data_path)
    if dev_table_path != None:
      self.dev_dict = self.get_table(dev_table_path)
      self.dev_data = self.get_data(self.dev_dict,dev_data_path)
  # get the dict of id->header
  def get_table(self,path): 
    id_header = {} 
    with open(path, 'r') as json_file:
      json_list = list(json_file)
    for json_str in json_list:
      result = json.loads(json_str)
      id_header[result['id']] = [result['header'],result['types'],result['rows']]
    return id_header

  def get_data(self,id_header,path):
    data = []
    with open(path, 'r') as json_file:
      json_list = list(json_file)

    for j in json_list:
      j = json.loads(j)
      dic = dict(j)
      dic["header"] = id_header[j['table_id']]
      data.append(dic)
    
    return data

# 模型

In [ ]:
class N2Sm1(nn.Module):
  def __init__(self,model_type):
    super(N2Sm1,self).__init__()
    config = BertConfig.from_pretrained(model_type)
    self.bert_model = BertModel.from_pretrained(model_type,config = config)
    self.cond_conn_op_decoder = nn.Linear(config.hidden_size,3)
    self.agg_deocder = nn.Linear(config.hidden_size,7)
    self.cond_op_decoder = nn.Linear(config.hidden_size,5)

  # 取得header_ids 所標記的 token
  def get_agg_hiddens(self,hiddens,header_ids):
    # header_ids [bsize,headers_idx]
    # hiddens [bsize,seqlength,worddim]
    arr = []
    for b_idx in range(0,hiddens.shape[0]):
      s = torch.stack([hiddens[b_idx][i] for i in header_ids[b_idx]],dim=0)
      arr.append(s)
    return torch.stack(arr,dim=0)

  def forward(self,input_ids=None,attention_mask=None,token_type_ids=None,header_ids=None): 
    # hidden [bsize,seqlength,worddim] cls [bsize,worddim]
    hiddens,cls = self.bert_model(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)
    cond_conn_op = self.cond_conn_op_decoder(cls)
    header_hiddens = self.get_agg_hiddens(hiddens,header_ids)
    agg = self.agg_deocder(header_hiddens)
    cond_op = self.cond_op_decoder(header_hiddens)
    # cond_conn_op [bsize,3]   
    # cond_op [bize,header_length,5]
    # agg [bize,header_length,7] 
    return cond_conn_op,cond_op , agg

In [ ]:
class N2Sm2(nn.Module):
  def __init__(self,model_type):
    super(N2Sm2,self).__init__()
    config = BertConfig.from_pretrained(model_type)
    self.bert_model = BertModel.from_pretrained(model_type,config = config)
    self.decoder = nn.Sequential(
        nn.Linear(config.hidden_size,1),
        nn.Sigmoid()
    )
    
  def forward(self,input_ids=None,attention_mask=None,token_type_ids=None,header_ids=None): 
    # hidden [bsize,seqlength,worddim] cls [bsize,worddim]
    hiddens,cls = self.bert_model(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)
    return self.decoder(cls)

# 抽數字

In [ ]:
def is_float(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

def cn_to_an(string):
    try:
        return str(cn2an.cn2an(string, 'normal'))
    except ValueError:
        return string

def an_to_cn(string):
    try:
        return str(cn2an.an2cn(string))
    except ValueError:
        return string

def str_to_num(string):
    try:
        float_val = float(cn_to_an(string))
        if int(float_val) == float_val:   
            return str(int(float_val))
        else:
            return str(float_val)
    except ValueError:
        return None

def str_to_year(string):
    year = string.replace('年', '')
    year = cn_to_an(year)
    if is_float(year) and float(year) < 1900:
        year = int(year) + 2000
        return str(year)
    else:
        return None
    
def load_json(json_file):
    result = []
    if json_file:
        with open(json_file) as file:
            for line in file:
                result.append(json.loads(line))
    return result

In [ ]:
def extract_values_from_text(text):
        values = []
        values += extract_year_from_text(text)
        values += extract_num_from_text(text)
        return list(set(values))

def extract_num_from_text(text):
    CN_NUM = '〇一二三四五六七八九零壹贰叁肆伍陆柒捌玖貮两'
    CN_UNIT = '十拾百佰千仟万萬亿億兆点'
    values = []
    num_values = re.findall(r'[-+]?[0-9]*\.?[0-9]+', text)
    values += num_values
        
    cn_num_unit = CN_NUM +CN_UNIT
    cn_num_texts = re.findall(r'[{}]*\.?[{}]+'.format(cn_num_unit, cn_num_unit), text)
    cn_num_values = [str_to_num(text) for text in cn_num_texts]
    values += [value for value in cn_num_values if value is not None]
    
    cn_num_mix = re.findall(r'[0-9]*\.?[{}]+'.format(CN_UNIT), text)
    for word in cn_num_mix:
        num = re.findall(r'[-+]?[0-9]*\.?[0-9]+', word)
        for n in num:
            word = word.replace(n, an_to_cn(n))
        str_num = str_to_num(word)
        if str_num is not None:
            values.append(str_num)
    return values


def extract_year_from_text(text):
  values = []
  CN_NUM = '〇一二三四五六七八九零壹贰叁肆伍陆柒捌玖貮两'
  CN_UNIT = '十拾百佰千仟万萬亿億兆点'
  num_year_texts = re.findall(r'[0-9][0-9]年', text)
  values += ['20{}'.format(text[:-1]) for text in num_year_texts]
  cn_year_texts = re.findall(r'[{}][{}]年'.format(CN_NUM,CN_NUM), text)
  cn_year_values = [str_to_year(text) for text in cn_year_texts]
  values += [value for value in cn_year_values if value is not None]
  return values

# 打包
+ M1 : (Question,headers)
+ M2 : (question,condition)
+ NL2SQL: (data)
+ data
  + table_id:
  + question: 呃共有多少家公司17年定增，而且是为了配套融资的呀
  + headers:[['h1','h2','h3'],['text','real','text']] 
  + table [['上海',1,'下雨'],[],[]...]

In [ ]:
class M1():
  def __init__(self,model_path,model_type,device): 
    
    # device
    self.device = device
    
    # model & tokenizer
    # model_type = 'hfl/chinese-wwm-ext'
    config = BertConfig.from_pretrained(model_type)
    self.tokenizer = BertTokenizer.from_pretrained(model_type)
    self.model = N2Sm1(model_type).to(self.device)
    self.model.load_state_dict(torch.load(model_path)) 

  def go(self,question,headers):
    # headerse_list : a list like: [['c1','c2'],['text','real']]
    # question  : 长沙2011年平均每天成交量是3.17，那么近一周的成交量是多少
    # 會回傳 conn_op:[0-2] agg: header_count * 7 cond header_count*5
    all_tokens = self.tokenizer.tokenize(question)
    col_type_token_dict = {'text': '[unused11]', 'real': '[unused12]'}
    for h,t in zip(headers[0],headers[1]):
      tokens = ['[SEP]',col_type_token_dict[t]] + self.tokenizer.tokenize(h)
      all_tokens = all_tokens + tokens
    # get the header token place 
    header_ids = []
    for i in range(len(all_tokens)):
      if all_tokens[i] == col_type_token_dict['text'] or all_tokens[i] == col_type_token_dict['real']:
        header_ids.append(i+1)
    plus = self.tokenizer.encode_plus(all_tokens)
    input_ids,token_type_ids,attention_mask,header_ids = torch.tensor(plus['input_ids']).unsqueeze(0)\
    ,torch.tensor(plus['token_type_ids']).unsqueeze(0),\
      torch.tensor(plus['attention_mask']).unsqueeze(0), torch.tensor(header_ids).unsqueeze(0)
    # pred
    cond_conn_op_pred,conds_ops_pred ,agg_pred = self.model(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids,header_ids=header_ids)
    result = {}
    result['conn_op'] = torch.argmax(cond_conn_op_pred.squeeze(),dim=-1).numpy()
    result['agg'] = torch.argmax(agg_pred.squeeze(),dim=-1).numpy()
    result['cond'] = torch.argmax(conds_ops_pred.squeeze(),dim=-1).numpy()
    
    return result


    
   

In [ ]:
class M2():
  def __init__(self,model_path,model_type,device): 
    
    # device
    self.device = device
    
    # model & tokenizer
    # model_type = 'hfl/chinese-wwm-ext'
    config = BertConfig.from_pretrained(model_type)
    self.tokenizer = BertTokenizer.from_pretrained(model_type)
    self.model = N2Sm2(model_type).to(self.device)
    self.model.load_state_dict(torch.load(model_path)) 
  def go(self,question,value):
    ids =  self.tokenizer.encode_plus(question,value)
    for k in ids.keys():
      ids[k] = torch.tensor(ids[k]).to(self.device)
    input_ids,token_type_ids,attention_mask = ids['input_ids'].unsqueeze(0),ids['token_type_ids'].unsqueeze(0),ids['attention_mask'].unsqueeze(0)
    pred = self.model(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids).squeeze().item()
    return pred


In [ ]:
# 終極版本
class NL2SQL():
  def __init__(self,model_type,device,m1_path,m2_path,analyze=False):
    # device
    self.device = device
    # model & tokenizer
    # model_type = 'hfl/chinese-wwm-ext'
    config = BertConfig.from_pretrained(model_type)
    self.tokenizer = BertTokenizer.from_pretrained(model_type)
    self.model_1 = N2Sm1(model_type).to(self.device)
    self.model_1.load_state_dict(torch.load(m1_path)) 
    self.model_2 = N2Sm2(model_type).to(self.device)
    self.model_2.load_state_dict(torch.load(m2_path))
    self.analyze = analyze
  def get_m1_output(self,question,headers):
    all_tokens = self.tokenizer.tokenize(question)
    col_type_token_dict = {'text': '[unused11]', 'real': '[unused12]'}
    for h,t in zip(headers[0],headers[1]):
      tokens = ['[SEP]',col_type_token_dict[t]] + self.tokenizer.tokenize(h)
      all_tokens = all_tokens + tokens
    # get the header token place 
    header_ids = []
    for i in range(len(all_tokens)):
      if all_tokens[i] == col_type_token_dict['text'] or all_tokens[i] == col_type_token_dict['real']:
        header_ids.append(i+1)
    plus = self.tokenizer.encode_plus(all_tokens)
    input_ids,token_type_ids,attention_mask,header_ids = torch.tensor(plus['input_ids']).unsqueeze(0).to(self.device)\
    ,torch.tensor(plus['token_type_ids']).unsqueeze(0).to(self.device),\
      torch.tensor(plus['attention_mask']).unsqueeze(0).to(self.device), torch.tensor(header_ids).unsqueeze(0).to(self.device)
    # pred
    cond_conn_op_pred,conds_ops_pred ,agg_pred = self.model_1(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids,header_ids=header_ids)
    result = {}
    result['conn_op'] = torch.argmax(cond_conn_op_pred.squeeze(),dim=-1).to('cpu').numpy()
    result['agg'] = torch.argmax(agg_pred.squeeze(),dim=-1).to('cpu').numpy()
    result['cond'] = torch.argmax(conds_ops_pred.squeeze(),dim=-1).to('cpu').numpy()
    return result
  def get_m2_output(self,question,cond):
    ids =  self.tokenizer.encode_plus(question,cond)
    for k in ids.keys():
      ids[k] = torch.tensor(ids[k]).to(self.device)
    input_ids,token_type_ids,attention_mask = ids['input_ids'].unsqueeze(0).to(self.device),ids['token_type_ids'].unsqueeze(0).to(self.device),ids['attention_mask'].unsqueeze(0).to(self.device)
    pred = self.model_2(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids).squeeze().item()
    return pred

  def get_sql(self,data,m1,table):
   # {'agg': array([6, 4, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]),
   # 'cond': array([4, 4, 4, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4]),
   # 'conn_op': array(1)}
    result = ''
    conn_map = ['','AND','OR']
    agg_map = ['','AVG','MAX','MIN','COUNT','SUM','']
    cond_map = ['>','<','=','!=','']
    agg,cond,conn_op = m1['agg'],m1['cond'],conn_map[m1['conn_op']]
    headers = data['headers']
    pre = ''   
    result += "SELECT "
    for col,val in enumerate(agg):
      if val!=6:
        result += pre+agg_map[val]+' '+headers[0][col]
        pre= ' , '
    # where condition       
    result += ' WHERE '
    pre = ''
    for col,val in enumerate(cond):
      if val!=4:
        if data['headers'][1][col] == 'text':
          values_list = set([r[col] for r in table]) # value from table
        elif data['headers'][1][col] == 'real':
          values_list = extract_values_from_text(data['question']) # value from question
          if self.analyze:
            print(values_list,'number from question')
        for v in values_list:
          cond = headers[0][col]+cond_map[val]+str(v)
          p = self.get_m2_output( data['question'],cond ) 
          if self.analyze:
            print(cond,p)
          if p > 0.5:
            result += pre + cond + ' '
            pre = conn_op+' '
       
    return result
  def go(self,data):
    # data:
    #   table_id:
    #   question:
    #   headers:[['h1','h2','h3'],['text','real','text']] 
    #   table [['上海',1,'下雨'],[],[]...]
    
    m1 = self.get_m1_output(data["question"],data["headers"])
    result = self.get_sql(data,m1,data["table"])
    return  result


# 操作

In [ ]:
path1 = './saved_models/M1.pt'
path2 = './saved_models/M2_05.pt'
model_type = 'hfl/chinese-bert-wwm'
model = NL2SQL(model_type,torch.device('cuda:0'),path1,path2,True)

In [ ]:
data = {
    'question': '呃共有多少家公司17年定增，而且是为了配套融资的呀', 
    'headers': [['证券代码', '证券简称', '最新收盘价', '定增价除权后至今价格', '增发价格', '倒挂率', '定增年度', '增发目的'], ['text', 'text', 'real', 'real', 'real', 'real', 'real', 'text']], 
    'table': [['002678.SZ', '珠江钢琴', 5.82, 9.37, 12.3, 62.09, 2017.0, '项目融资'], 
          ['002696.SZ', '百洋股份', 8.87, 10.53, 18.19, 84.26, 2017.0, '配套融资'], 
          ['002696.SZ', '百洋股份', 8.87, 12.0, 20.73, 73.93, 2017.0, '融资收购其他资产'], 
          ['300044.SZ', '赛为智能', 6.62, 7.24, 13.06, 91.49, 2017.0, '融资收购其他资产'], 
          ['300050.SZ', '世纪鼎利', 5.6, 12.64, 12.68, 44.3, 2017.0, '配套融资'], 
          ['300050.SZ', '世纪鼎利', 5.6, 12.22, 12.26, 45.82, 2017.0, '融资收购其他资产']]
    }

In [ ]:
model.go(data)

['2017', '17'] number from question
定增年度=2017 0.9999343156814575
定增年度=17 0.00010633746569510549
增发目的=项目融资 8.339100168086588e-05
增发目的=配套融资 0.9999605417251587
增发目的=融资收购其他资产 8.809728024061769e-05


'SELECT COUNT 证券简称 WHERE 定增年度=2017 AND 增发目的=配套融资 '

In [ ]:
train_table_file = './data/train/train.tables.json'
train_data_file = './data/train/train.json'

val_table_file = './data/val/val.tables.json'
val_data_file = './data/val/val.json'

test_table_file = './data/test/test.tables.json'
test_data_file = './data/test/test.json'
sample_datas = T2Sdataset(train_table_file,train_data_file,test_table_file,test_data_file,val_table_file,val_data_file)

In [ ]:
import random
idx = random.randrange(4396)
d = sample_datas.dev_data[idx]
print(d['sql'])
data = {
    "table_id": d["table_id"],
    "question": d["question"],
    "headers": d["header"][0:2],
    "table": d["header"][2]
}
print(data)
print('=================================\n')
print('正解:' ,d['sql'])
print('Header:' , data['headers'][0])
print('Q:' , data['question'],'\n')
print('預測SQL: ' ,model.go(data))


{'agg': [4], 'cond_conn_op': 1, 'sel': [1], 'conds': [[6, 2, '2017'], [7, 2, '配套融资']]}
{'table_id': '4d263cee3aaa11e989cef40f24344a08', 'question': '呃共有多少家公司17年定增，而且是为了配套融资的呀', 'headers': [['证券代码', '证券简称', '最新收盘价', '定增价除权后至今价格', '增发价格', '倒挂率', '定增年度', '增发目的'], ['text', 'text', 'real', 'real', 'real', 'real', 'real', 'text']], 'table': [['002678.SZ', '珠江钢琴', 5.82, 9.37, 12.3, 62.09, 2017.0, '项目融资'], ['002696.SZ', '百洋股份', 8.87, 10.53, 18.19, 84.26, 2017.0, '配套融资'], ['002696.SZ', '百洋股份', 8.87, 12.0, 20.73, 73.93, 2017.0, '融资收购其他资产'], ['300044.SZ', '赛为智能', 6.62, 7.24, 13.06, 91.49, 2017.0, '融资收购其他资产'], ['300050.SZ', '世纪鼎利', 5.6, 12.64, 12.68, 44.3, 2017.0, '配套融资'], ['300050.SZ', '世纪鼎利', 5.6, 12.22, 12.26, 45.82, 2017.0, '融资收购其他资产']]}

正解: {'agg': [4], 'cond_conn_op': 1, 'sel': [1], 'conds': [[6, 2, '2017'], [7, 2, '配套融资']]}
Header: ['证券代码', '证券简称', '最新收盘价', '定增价除权后至今价格', '增发价格', '倒挂率', '定增年度', '增发目的']
Q: 呃共有多少家公司17年定增，而且是为了配套融资的呀 

['17', '2017'] v
定增年度=17 0.0004996743518859148
定增年度=2017 0

# test

In [ ]:
import re

In [ ]:
class CandidateCondsExtractor():
    """
    params:
        - share_candidates: 在同 table 同 column 中共享 real 型 candidates
    """
    CN_NUM = '〇一二三四五六七八九零壹贰叁肆伍陆柒捌玖貮两'
    CN_UNIT = '十拾百佰千仟万萬亿億兆点'
    
    def __init__(self, share_candidates=True):
        self.share_candidates = share_candidates
        self._cached = False
    
    def build_candidate_cache(self, queries):
        self.cache = defaultdict(set)
        print('building candidate cache')
        for query_id, query in tqdm(enumerate(queries), total=len(queries)):
            value_in_question = self.extract_values_from_text(query.question.text)
            
            for col_id, (col_name, col_type) in enumerate(query.table.header):
                value_in_column = self.extract_values_from_column(query, col_id)
                if col_type == 'text':
                    cond_values = value_in_column
                elif col_type == 'real':
                    if len(value_in_column) == 1: 
                        cond_values = value_in_column + value_in_question
                    else:
                        cond_values = value_in_question
                cache_key = self.get_cache_key(query_id, query, col_id)
                self.cache[cache_key].update(cond_values)
        self._cached = True
    
    def get_cache_key(self, query_id, query, col_id):
        if self.share_candidates:
            return (query.table.id, col_id)
        else:
            return (query_id, query.table.id, col_id)
        
    def extract_year_from_text(self, text):
        values = []
        num_year_texts = re.findall(r'[0-9][0-9]年', text)
        values += ['20{}'.format(text[:-1]) for text in num_year_texts]
        cn_year_texts = re.findall(r'[{}][{}]年'.format(self.CN_NUM, self.CN_NUM), text)
        cn_year_values = [str_to_year(text) for text in cn_year_texts]
        values += [value for value in cn_year_values if value is not None]
        return values
    
    def extract_num_from_text(self, text):
        values = []
        num_values = re.findall(r'[-+]?[0-9]*\.?[0-9]+', text)
        values += num_values
        
        cn_num_unit = self.CN_NUM + self.CN_UNIT
        cn_num_texts = re.findall(r'[{}]*\.?[{}]+'.format(cn_num_unit, cn_num_unit), text)
        cn_num_values = [str_to_num(text) for text in cn_num_texts]
        values += [value for value in cn_num_values if value is not None]
    
        cn_num_mix = re.findall(r'[0-9]*\.?[{}]+'.format(self.CN_UNIT), text)
        for word in cn_num_mix:
            num = re.findall(r'[-+]?[0-9]*\.?[0-9]+', word)
            for n in num:
                word = word.replace(n, an_to_cn(n))
            str_num = str_to_num(word)
            if str_num is not None:
                values.append(str_num)
        return values
    
    def extract_values_from_text(self, text):
        values = []
        values += self.extract_year_from_text(text)
        values += self.extract_num_from_text(text)
        return list(set(values))
   
    def extract_values_from_column(self, query, col_ids):
        question = query.question.text
        question_chars = set(query.question.text)
        unique_col_values = set(query.table.df.iloc[:, col_ids].astype(str))
        select_col_values = [v for v in unique_col_values 
                             if (question_chars & set(v))]
        return select_col_values

In [ ]:
def is_float(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

def cn_to_an(string):
    try:
        return str(cn2an.cn2an(string, 'normal'))
    except ValueError:
        return string

def an_to_cn(string):
    try:
        return str(cn2an.an2cn(string))
    except ValueError:
        return string

def str_to_num(string):
    try:
        float_val = float(cn_to_an(string))
        if int(float_val) == float_val:   
            return str(int(float_val))
        else:
            return str(float_val)
    except ValueError:
        return None

def str_to_year(string):
    year = string.replace('年', '')
    year = cn_to_an(year)
    if is_float(year) and float(year) < 1900:
        year = int(year) + 2000
        return str(year)
    else:
        return None
    
def load_json(json_file):
    result = []
    if json_file:
        with open(json_file) as file:
            for line in file:
                result.append(json.loads(line))
    return result

In [ ]:
def extract_values_from_text(text):
        values = []
        values += extract_year_from_text(text)
        values += extract_num_from_text(text)
        return list(set(values))

def extract_num_from_text(text):
    CN_NUM = '〇一二三四五六七八九零壹贰叁肆伍陆柒捌玖貮两'
    CN_UNIT = '十拾百佰千仟万萬亿億兆点'
    values = []
    num_values = re.findall(r'[-+]?[0-9]*\.?[0-9]+', text)
    values += num_values
        
    cn_num_unit = CN_NUM +CN_UNIT
    cn_num_texts = re.findall(r'[{}]*\.?[{}]+'.format(cn_num_unit, cn_num_unit), text)
    cn_num_values = [str_to_num(text) for text in cn_num_texts]
    values += [value for value in cn_num_values if value is not None]
    
    cn_num_mix = re.findall(r'[0-9]*\.?[{}]+'.format(CN_UNIT), text)
    for word in cn_num_mix:
        num = re.findall(r'[-+]?[0-9]*\.?[0-9]+', word)
        for n in num:
            word = word.replace(n, an_to_cn(n))
        str_num = str_to_num(word)
        if str_num is not None:
            values.append(str_num)
    return values


def extract_year_from_text(text):
  values = []
  CN_NUM = '〇一二三四五六七八九零壹贰叁肆伍陆柒捌玖貮两'
  CN_UNIT = '十拾百佰千仟万萬亿億兆点'
  num_year_texts = re.findall(r'[0-9][0-9]年', text)
  values += ['20{}'.format(text[:-1]) for text in num_year_texts]
  cn_year_texts = re.findall(r'[{}][{}]年'.format(CN_NUM,CN_NUM), text)
  cn_year_values = [str_to_year(text) for text in cn_year_texts]
  values += [value for value in cn_year_values if value is not None]
  return values

In [ ]:
q = '播放量破一千万的动漫有多少部啊？'
extract_values_from_text(q)

['10000000']

In [ ]:
path1 = './saved_models/M1.pt'
path2 = './saved_models/M2_04.pt'
model_type = 'hfl/chinese-bert-wwm'

model1 = M1(path1,model_type,torch.device('cpu'))
model2 = M2(path2,model_type,torch.device('cpu'))

In [ ]:
train_table_file = './data/train/train.tables.json'
train_data_file = './data/train/train.json'

val_table_file = './data/val/val.tables.json'
val_data_file = './data/val/val.json'

test_table_file = './data/test/test.tables.json'
test_data_file = './data/test/test.json'
sample_datas = T2Sdataset(train_table_file,train_data_file,test_table_file,test_data_file,val_table_file,val_data_file)

In [ ]:
idx = 554
d = sample_datas.dev_data[idx]
data = {
    "table_id": d["table_id"],
    "question": d["question"],
    "headers": d["header"][0:2],
    "table": d["header"][2]
}
print(data)

{'table_id': '015637de351311e9a016542696d6e445', 'question': 'EPS2017A大于0.5并且EPS2018E大于0.5的证券总共有多少？', 'headers': [['代码', '证券', '收盘价', 'EPS2017A', 'EPS2018E', 'EPS2019E', 'EPS2020E', 'PE2017A', 'PE2018E', 'PE2019E', 'PE2020E', 'CAGR18~20', 'PEG'], ['real', 'text', 'real', 'real', 'real', 'real', 'real', 'real', 'real', 'real', 'real', 'real', 'real']], 'table': [[601111.0, '中国国航', 8.13, 0.54, 0.44, 0.7, 0.91, 15.0, 19.0, 12.0, 9.0, 18.86, 0.99], [600115.0, '东方航空', 5.1, 0.44, 0.27, 0.54, 0.75, 12.0, 19.0, 9.0, 7.0, 19.61, 0.97], [601021.0, '春秋航空', 31.73, 1.58, 1.68, 2.2, 2.8, 20.0, 19.0, 14.0, 11.0, 21.01, 0.9], [603885.0, '吉祥航空', 11.86, 0.74, 0.82, 1.0, 1.3, 16.0, 15.0, 12.0, 9.0, 20.8, 0.7], [600029.0, '南方航空', 7.18, 0.48, 0.31, 0.76, 1.05, 15.0, 23.0, 9.0, 7.0, 29.81, 0.78]]}


In [ ]:
model1.go(data['question'],data['headers'])

{'agg': array([6, 4, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]),
 'cond': array([4, 4, 4, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4]),
 'conn_op': array(1)}

In [ ]:
q = '2010成交面积大于500而且2011年成交面积大于500的一线城市累计同比最大是多少？'
v  = '2011年成交面积>2011'
model2.go(q,v)

0.00017600416322238743

In [ ]:
data['question']="18年的拉菲"
ls = extract_values_from_text(data['question'])
print(ls)
model2.go(data['question'],v)

['2018', '18']


0.04116003215312958

# 驗證

In [ ]:
train_table_file = './data/train/train.tables.json'
train_data_file = './data/train/train.json'

val_table_file = './data/val/val.tables.json'
val_data_file = './data/val/val.json'

test_table_file = './data/test/test.tables.json'
test_data_file = './data/test/test.json'
sample_datas = T2Sdataset(train_table_file,train_data_file,test_table_file,test_data_file,val_table_file,val_data_file)
print(len(sample_datas.dev_data))

4396


In [ ]:
path1 = './saved_models/M1.pt'
path2 = './saved_models/M2_04.pt'
model_type = 'hfl/chinese-bert-wwm'

model1 = M1(path1,model_type,torch.device('cpu'))

In [ ]:

agg_correct = 0
conn_correct = 0
sel_correct = 0
cond_correct = 0
for ii,data in enumerate(sample_datas.dev_data):
  sql = data['sql']
  result = model1.go(data['question'],data['header'])
  # print(idx,sql,result)
  # conn
  if sql['cond_conn_op'] == result['conn_op']:
    conn_correct = conn_correct + 1
  # sel  
  #print('sel',np.where(result['agg']!=6)[0].tolist(), sql['sel'])  
  if np.where(result['agg']!=6)[0].tolist() == sql['sel']:
    sel_correct = sel_correct + 1

  # agg
  idx = np.where(result['agg']!=6)[0].tolist()
  #print('agg', [ result['agg'][i] for i in idx] , sql['agg'])
  if [ result['agg'][i] for i in idx] == sql['agg']: 
    agg_correct += 1
  
  # cond 
 
  all_right = True
  for cond in sql['conds']:
   # print(result['cond'][cond[0]] , cond[1] ,result['cond'][cond[0]]==cond[1]  )
    if result['cond'][cond[0]] != cond[1]:
      all_right = False
      break
  if all_right:
    cond_correct+=1 

In [ ]:
print(agg_correct ,conn_correct ,sel_correct,cond_correct)
print(agg_correct/4396 ,conn_correct/4396 ,sel_correct/4396,cond_correct/4396)

4194 4232 4097 4050
0.954049135577798 0.9626933575978162 0.9319836214740673 0.9212920837124658
